<a href="https://colab.research.google.com/github/HeNeos/Mechanical/blob/master/Finite_Element_Analysis/Python_code_for_%22Matlab-Guide-to-Finite-Elements%22/Plane_truss_element.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

In [13]:
def NaiveMultiply(A,B):
    C = np.zeros((A.shape[0],B.shape[1]))
    for i in range(0,C.shape[0]):
        for j in range(0,C.shape[1]):
            aux = 0
            for k in range(0,A.shape[1]):
                aux += A[i][k]*B[k][j]
            C[i][j] = aux
    return C
 
def nextPowerofTwo(n):
    return int(2**(ceil(log2(n))))
 
def ModifyMatrix(A):
    newA = A
    if(A.shape[1]%2 == 1):
        aux = np.zeros((A.shape[0],A.shape[1]+1))
        aux = np.insert(A,A.shape[1],0,axis=1)
        newA = aux
    if(newA.shape[0]%2 == 1):
        aux = np.zeros((newA.shape[0]+1,newA.shape[1]))
        aux = np.insert(newA,newA.shape[0],0,axis=0)
        newA = aux
    return newA
 
def FastMultiply(oldA,oldB):
    rows = oldA.shape[0]
    columns = oldB.shape[1]
    if(rows <= 2 or columns <=2 or oldA.shape[1] <= 2 or oldB.shape[0] <= 2):
        return np.matmul(oldA,oldB)
    
    A = ModifyMatrix(oldA)
    B = ModifyMatrix(oldB)
    N1 = A.shape[0]
    N2 = A.shape[1]
    N3 = B.shape[0]
    N4 = B.shape[1]
    
 
    a = A[0:N1//2,0:N2//2]
    b = A[0:N1//2,N2//2:N2//2+N2//2]
    c = A[N1//2:N1//2+N1//2,0:N2//2]
    d = A[N1//2:N1//2+N1//2,N2//2:N2//2+N2//2]
    
    e = B[0:N3//2,0:N4//2]
    f = B[0:N3//2,N4//2:N4//2+N4//2]
    g = B[N3//2:N3//2+N3//2,0:N4//2]
    h = B[N3//2:N3//2+N3//2,N4//2:N4//2+N4//2]
    
    
    p1 = FastMultiply(a,(f-h))
    p3 = FastMultiply((c+d),e)
    p2 = FastMultiply((a+b),h)
    p4 = FastMultiply(d,(g-e))
    p5 = FastMultiply((a+d),(e+h))
    p6 = FastMultiply((b-d),(g+h))
    p7 = FastMultiply((a-c),(e+f))
    
    
    C = np.zeros((rows,columns))
    
    c11 = p5 + p4 - p2 + p6
    c12 = p1 + p2
    c21 = p3 + p4
    c22 = p1 + p5 - p3 - p7
    
 
    
    for i in range(0,N1//2):
        for j in range(0,N4//2):
            C[i][j] = c11[i][j]
            if(j + N4//2 < columns):
                C[i][j+N4//2] = c12[i][j]
            if(i + N1//2 < rows):
                C[i+N1//2][j] = c21[i][j]
                if(j + N4//2 < columns):
                    C[i+N1//2][j+N4//2] = c22[i][j]
    return C

In [14]:
def conjugate_grad(A, b, x=None):
    n = b.shape[0]
    if not x:
        x = np.ones((n,1))
    r = np.dot(A, x) - b
    p = - r
    r_k_norm = FastMultiply(np.transpose(r), r)
    for i in range(2*n):
        Ap = np.dot(A, p)
        
        alpha = r_k_norm /FastMultiply(np.transpose(p), Ap)
        x += alpha * p
        r += alpha * Ap
        r_kplus1_norm = FastMultiply(np.transpose(r), r)
        beta = r_kplus1_norm / r_k_norm
        r_k_norm = r_kplus1_norm
        p = beta * p - r
    return x

In [40]:
NodesCondition = []
ForcesCondition = []

def DistNodes(f,s):
    if(s[0] == f[0]):
        aux = np.pi/2
        if(s[1] < f[1]):
             aux *= -1
        return (np.sqrt((s[0]-f[0])**2+(s[1]-f[1])**2),aux)
    else:
        aux = np.arctan((s[1]-f[1])/(s[0]-f[0]))
    if(aux < 0 and s[1] > f[1]):
        aux += np.pi
    if(s[1] < f[1]):
        aux += np.pi
        if(s[0] > f[0]):
            aux += np.pi

    return (np.sqrt((s[0]-f[0])**2+(s[1]-f[1])**2),aux)
 
def UBoundaryCondition(nU,u,i):
    nU[i][0] = u
    NodesCondition.append(i)
 
def FBoundaryCondition(nF,f,i):
    nF[i][0] += f
    ForcesCondition.append(i)
    
def AssemblyStiffness(nStiffnessMatrix,k,i,j):
    for p in range(0,2):
        for m in range(0,2):
            nStiffnessMatrix[2*i+p][2*i+m] += k[p][m]
            nStiffnessMatrix[2*i+p][2*j+m] += k[p][2+m]
            nStiffnessMatrix[2*j+p][2*i+m] += k[p+2][m]
            nStiffnessMatrix[2*j+p][2*j+m] += k[p+2][2+m]
 
def Initialize(nStiffnessMatrix,nU,nF):
    for i in range(0,Nodes):
        nU[i][0] = 0
        nF[i][0] = 0
    for i in range(0,NumberOfElement):
        AssemblyStiffness(nStiffnessMatrix,K[i],int(Elements[i][0]),int(Elements[i][1]))

def TMatrix(nT,i,angle):
    nT[2*i][2*i] = np.cos(angle)
    nT[2*i][2*i+1] = np.sin(angle)
    nT[2*i+1][2*i] = -np.sin(angle)
    nT[2*i+1][2*i+1] = np.cos(angle)

def ApplyT(nStiffnessMatrix, nT):
    return FastMultiply(FastMultiply(nT,nStiffnessMatrix),np.transpose(nT))

def PreSolvingStiffness(nStiffnessMatrix):
    nsize = Nodes-len(NodesCondition)
    newStiffness = np.zeros((nsize,nsize))
    contr = -1
    for i in range(0,Nodes):
        contc = -1
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        for j in range(0,Nodes):
            flagc = False
            for k in range(0,len(NodesCondition)):
                if(j == NodesCondition[k]):
                    flagc = True
                    break
            if(flagc):
                continue
            contc += 1
            newStiffness[contr][contc] = nStiffnessMatrix[i][j]
    return newStiffness
 
 
def PreSolvingF(nF,nS,nU):
    nsize = Nodes-len(NodesCondition)
    newF = np.zeros(nsize).reshape(nsize,1)
    contr = -1
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            for k in range(0,Nodes):
                nF[k][0] = nF[k][0]-nS[k][i]*nU[i][0]
            continue
 
            
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        newF[contr][0] = nF[i][0]
    
    return newF
                      
 
def Solve(nStiffnessMatrix,nU,nF):
    newStiffness = PreSolvingStiffness(nStiffnessMatrix)
    newF = PreSolvingF(nF,nStiffnessMatrix,nU)
    u = conjugate_grad(newStiffness,newF)    
    contr = -1
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        nU[i][0] = u[contr][0]
    nnF = FastMultiply(StiffnessMatrix,nU)
    return nU,nnF

In [41]:
NodesCondition = []
Nodes = 3
Nodes *= 2
NumberOfElement = 3

E = 210e6 #MPA
K = []
A = 1e-4*np.ones(Nodes)
PosNodes = np.array([(0,0),(4,0),(2,3)])
L = np.array([DistNodes(PosNodes[0],PosNodes[1]),DistNodes(PosNodes[0],PosNodes[2]),DistNodes(PosNodes[1],PosNodes[2])])
Elements = np.array([(0,1),(0,2),(1,2)])


for i in range(0,NumberOfElement):
    aux = np.zeros((4,4))
    angle = L[i][1]
    rows = [np.cos(angle),np.sin(angle),-np.cos(angle),-np.sin(angle)]
    cols = [np.cos(angle),np.sin(angle),-np.cos(angle),-np.sin(angle)]
    for j in range(0,4):
        for k in range(0,4):
            aux[j][k] = rows[j]*cols[k]
    aux = aux*E*A[i]/L[i][0]
    K.append(aux)


StiffnessMatrix = np.zeros((Nodes,Nodes))

U = np.zeros(Nodes).reshape(Nodes,1)
F = np.zeros(Nodes).reshape(Nodes,1)

Initialize(StiffnessMatrix,U,F)

#Node in UBoundary = Node*2+(x=0,y=1)
UBoundaryCondition(U,0,2*0+0) #Nodo 0 en X
UBoundaryCondition(U,0,2*0+1) #Nodo 0 en Y
UBoundaryCondition(U,0,2*1+1) #Nodo 1 en Y

FBoundaryCondition(F,0,2*1+0) #Nodo 1 en X
FBoundaryCondition(F,5,2*2+0) #Nodo 2 en X
FBoundaryCondition(F,-10,2*2+1) #Nodo 2 en Y

U,F=Solve(StiffnessMatrix,U,F)
print("Stiffness Matrix:\n",StiffnessMatrix,'\n')
print("Displacements:\n",U,'\n')
print("Forces:\n",F)

Stiffness Matrix:
 [[ 7.04210833e+03  2.68816249e+03 -5.25000000e+03  0.00000000e+00
  -1.79210833e+03 -2.68816249e+03]
 [ 2.68816249e+03  4.03224373e+03  0.00000000e+00  0.00000000e+00
  -2.68816249e+03 -4.03224373e+03]
 [-5.25000000e+03  0.00000000e+00  7.04210833e+03 -2.68816249e+03
  -1.79210833e+03  2.68816249e+03]
 [ 0.00000000e+00  0.00000000e+00 -2.68816249e+03  4.03224373e+03
   2.68816249e+03 -4.03224373e+03]
 [-1.79210833e+03 -2.68816249e+03 -1.79210833e+03  2.68816249e+03
   3.58421665e+03  9.09494702e-13]
 [-2.68816249e+03 -4.03224373e+03  2.68816249e+03 -4.03224373e+03
   9.09494702e-13  8.06448747e+03]] 

Displacements:
 [[ 0.        ]
 [ 0.        ]
 [ 0.00111111]
 [ 0.        ]
 [ 0.00195056]
 [-0.00161037]] 

Forces:
 [[-5.00000000e+00]
 [ 1.25000000e+00]
 [ 1.86517468e-14]
 [ 8.75000000e+00]
 [ 5.00000000e+00]
 [-1.00000000e+01]]


In [42]:
ElementDisplacement = []
for i in range(0,NumberOfElement):
    Node1 = Elements[i][0]
    Node2 = Elements[i][1]
    ElementDisplacement.append([(U[2*Node1][0],U[2*Node1+1][0]),(U[2*Node2][0],U[2*Node2+1][0])])
Stress = []
for i in range(0,NumberOfElement):
    C = np.cos(L[i][1])
    S = np.sin(L[i][1])
    dd = ElementDisplacement[i]
    ss = -dd[0][0]*C - dd[0][1]*S + dd[1][0]*C+dd[1][1]*S
    Stress.append(ss*E/L[i][0])
Stress = 1e-3*np.array(Stress)
Stress

array([  58.33333333,  -15.02313031, -105.1619122 ])

In [43]:
NodesCondition = []
Nodes = 4
Nodes *= 2
NumberOfElement = 6

E = 70e6 #MPA
K = []
A = 0.004*np.ones(Nodes)

PosNodes = np.array([(0,0),(0,3.5),(4,3.5),(4,0)])
L = np.array([DistNodes(PosNodes[0],PosNodes[1]),DistNodes(PosNodes[0],PosNodes[2]),
              DistNodes(PosNodes[0],PosNodes[3]),DistNodes(PosNodes[1],PosNodes[2]),
              DistNodes(PosNodes[1],PosNodes[3]),DistNodes(PosNodes[2],PosNodes[3])])

Elements = np.array([(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)])


for i in range(0,NumberOfElement):
    aux = np.zeros((4,4))
    angle = L[i][1]
    rows = [np.cos(angle),np.sin(angle),-np.cos(angle),-np.sin(angle)]
    cols = [np.cos(angle),np.sin(angle),-np.cos(angle),-np.sin(angle)]
    for j in range(0,4):
        for k in range(0,4):
            aux[j][k] = rows[j]*cols[k]
    aux = aux*E*A[i]/L[i][0]
    K.append(aux)


StiffnessMatrix = np.zeros((Nodes,Nodes))
T = np.eye(Nodes)
TMatrix(T,3,45*np.pi/180)

U = np.zeros(Nodes).reshape(Nodes,1)
F = np.zeros(Nodes).reshape(Nodes,1)

Initialize(StiffnessMatrix,U,F)

StiffnessMatrix = ApplyT(StiffnessMatrix,T)

#Node in UBoundary = Node*2+(x=0,y=1)
UBoundaryCondition(U,0,2*0+0) #Nodo 0 en X
UBoundaryCondition(U,0,2*0+1) #Nodo 0 en Y
UBoundaryCondition(U,0,2*3+1) #Nodo 3 en Y

FBoundaryCondition(F,0,2*1+0) #Nodo 1 en X
FBoundaryCondition(F,0,2*1+1) #Nodo 1 en Y
FBoundaryCondition(F,0,2*2+1) #Nodo 2 en Y
FBoundaryCondition(F,0,2*3+0) #Nodo 3 en X
FBoundaryCondition(F,30,2*2+0) #Nodo 2 en X


U,F=Solve(StiffnessMatrix,U,F)
print("Stiffness Matrix:\n",StiffnessMatrix,'\n')
print("Displacements:\n",U,'\n')
print("Forces:\n",F)

Stiffness Matrix:
 [[ 9.98366690e+04  2.61070853e+04 -2.91038305e-11  1.45519152e-11
  -2.98366690e+04 -2.61070853e+04 -4.94974747e+04  4.94974747e+04]
 [ 2.61070853e+04  1.02843700e+05  1.45519152e-11 -8.00000000e+04
  -2.61070853e+04 -2.28436997e+04  0.00000000e+00  1.45519152e-11]
 [-1.45519152e-11  0.00000000e+00  9.98366690e+04 -2.61070853e+04
  -7.00000000e+04 -3.63797881e-12 -2.63721387e+03  3.95582080e+04]
 [-1.81898940e-11 -8.00000000e+04 -2.61070853e+04  1.02843700e+05
  -3.63797881e-12  0.00000000e+00  2.30756213e+03 -3.46134320e+04]
 [-2.98366690e+04 -2.61070853e+04 -7.00000000e+04  4.89858720e-12
   9.98366690e+04  2.61070853e+04  1.45519152e-11  2.37737042e-12]
 [-2.61070853e+04 -2.28436997e+04  4.89858720e-12  1.45519152e-11
   2.61070853e+04  1.02843700e+05 -5.65685425e+04 -5.65685425e+04]
 [-4.94974747e+04 -1.45519152e-11 -2.63721387e+03  2.30756213e+03
  -1.45519152e-11 -5.65685425e+04  7.52330990e+04  1.50351536e+03]
 [ 4.94974747e+04  7.27595761e-12  3.95582080e+04 

In [44]:
ElementDisplacement = []
for i in range(0,NumberOfElement):
    Node1 = Elements[i][0]
    Node2 = Elements[i][1]
    ElementDisplacement.append([(U[2*Node1][0],U[2*Node1+1][0]),(U[2*Node2][0],U[2*Node2+1][0])])
Stress = []
for i in range(0,NumberOfElement):
    C = np.cos(L[i][1])
    S = np.sin(L[i][1])
    dd = ElementDisplacement[i]
    ss = -dd[0][0]*C - dd[0][1]*S + dd[1][0]*C+dd[1][1]*S
    Stress.append(ss*E/L[i][0])
Stress = 1e-3*np.array(Stress)
Stress

array([ 3.17912018,  5.13797441, -4.14254236,  3.6332802 , -6.96662758,
       -6.73105959])